# 1. 普通文件数据同步

- NFS网络文件共享可以同步存储数据。
- Samba共享数据：https://blog.51cto.com/taokey/1203553。
- 定时任务或守护进程结合。
- Inotify+rsync触发式实时数据同步。
- ftp数据同步。http://down.51cto.com/data/1094997。
- ssh key+scp/rsync。
- svn版本管理。
- rsync，sersync，inotify，union(双向同步)，csync(多向同步)。

# 2. Linux运维场景数据同步方案

## 2.1 scp，NFS，sftp，http，samba，rsync，csync，union
思想：
1. 文件级别也可以利用mysql，mongodb等软件。
2. 两个服务器同时写数据，双写就是一个同步机制。

## 2.2 文件系统级别同步
drdb（基于文件系统同步网络RAID1），同步几乎任何业务数据。

mysql数据库的官方推荐drdb同步数据，所有单点服务例如:NFS，MFS(DRBD)等都可以用drdb。

## 2.3 数据库同步方案
1. 自动同步机制：

Mysql replication，mysql 主从复制（逻辑的SQL从写）。

Oracle dataguard(物理的磁盘块，逻辑的sql语句从写)。

2. 第三方drdb

https://blog.51cto.com/oldboy/1240412。

# 3. MySQL主从复制

## 3.1 mysql主从复制介绍
MySQL支持单向，双向，链式级联，实时、异步同步，在复制过程中一台服务器充当主服务器（master），而一个或多个其他的服务器充当从服务器（slave）。

复制可以是单向：M ==> S，也可以是双向M <==> M，当然也可以多M环状同步等。  

如果设置了链式级联复制，那么从（slave）服务器本身除了充当从服务器外，也会同时充当其下面从服务器的主服务器。

链式级联复制类似A->B->C->D的复制形式。

#### 1. 单向主从同步逻辑图

<img src='./files/images/master_slave.png' width=400/>
<img src='./files/images/master_slaves.png' width=400/>

#### 2. 双向主主同步逻辑图

<img src='./files/images/double_masters.png' width=400/>

#### 3. 线性级联单向双主同步逻辑图

<img src='./files/images/series_master.png' width=400/>

#### 4. 环状级联单向多主同步

<img src='./files/images/circle_master.png' width=400/>

#### 5. 环状级联单向多主多从同步逻辑图

<img src='./files/images/circle_masters.png' width=400/>

<img src='./files/images/mysql001.png' width=400/>

在生产环境中所有对数据库内容的更新，就必须在主服务器上进行，以避免用户对主服务器上数据内容的更新与对从服务器上数据库内容的更新不一致而导致发生冲突。

## 3.2 主从复制的应用场景

mysql主从复制有利于数据库架构的健壮性，提升访问速度和易于维护管理。

### 1. 主从服务器**互为备份**

主从服务器架构的设置，可以大大的加强数据库架构的健壮性，例如当主服务器出现问题时，我们可以人工或自动切换到从服务器继续提供服务。

对于非人为的硬件，服务故障，对于人为的执行drop，delete无能为力。

### 2. 主从服务器**读写分离**分担网站压力

主从服务器架构可通过程序（php,java）或代理软件（mysql-proxy,amoeba）对用户（客户端）的请求实现**读写分离**，即通过在**从库上仅仅处理用户的select查询请求**，降低用户查询响应时间及读写请求对主服务器的压力，对于更新的数据（update，insert，delete）仍然交给主服务器处理，确保主服务器和从服务器保持实时同步。

中大型公司：通过程序（php,java）。  

测试环境：代理软件（mysql-proxy,amoeba）。  

门户网站：分布式dbproxy(读写分离，hash负载均衡，健康检查)。  

### 3. 根据服务器拆分业务独立并分担压力

可以把几个不同的从服务器，根据公司的业务进行拆分。

例如：
- 有为用户提供查询功能的从服务器，
- 有DBA用来备份的从服务器，
- 还有提供公司内部人员访问的后台，脚本，日志分析及开发人员服务的从服务器      
这样的拆分除了减轻主服务器的压力外使得对外用户浏览，对内处理公司内部用户业务，及DBA备份业务互不受影响。

主从架构生产环境从服务器分业务拆分使用案例：

M—>S1 ==>对外部用户提供服务（浏览帖子、浏览博客、浏览文章）负载均衡。

-->S2 ==>对外部用户提供服务（浏览帖子、浏览博客、浏览文章）负载均衡。

-->S3 ==>对外部用户提供服务（浏览帖子、浏览博客、浏览文章）负载均衡

-->S4 ==>对内部用户提供服务（后台访问、脚本任务、数据分析、开发人员浏览）

-->S5==>数据库备份服务（开启从服务器binlog功能，可实现增量备份及恢复）

逻辑图：